In [1]:
!cd /home/kamil.kural/FdaPrecision Challenge

/usr/bin/sh: line 0: cd: /home/kamil.kural/FdaPrecision: No such file or directory


In [2]:
import os
current_path = os.getcwd()

In [3]:
current_path

'/home/kamil.kural/FdaPrecision Challenge'

In [4]:
import numpy as np
import pandas as pd
import os
import copy
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15
from collections import defaultdict
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import eli5
import shap
from IPython.display import HTML
import json
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
from typing import List

import os
import time
import datetime
import json
import gc
from numba import jit

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn import metrics
from typing import Any
from itertools import product
pd.set_option('max_rows', 500)
import re
from tqdm import tqdm
from joblib import Parallel, delayed

Using TensorFlow backend.


In [5]:
from sklearn.tree import export_graphviz
import xgboost
import lightgbm
from sklearn.metrics import roc_auc_score

In [6]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from IPython.display import display
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
import torch 
import fastai
import bayes_opt

In [7]:
df1=pd.read_csv ('/home/kamil.kural/FdaPrecision Challenge/sc3_Phase1_CN_GE_FeatureMatrix.tsv', sep='\t')

In [8]:
df2=pd.read_csv ('/home/kamil.kural/FdaPrecision Challenge/sc3_Phase1_CN_GE_Outcome.tsv', sep='\t')

In [9]:
df3=pd.read_csv ('/home/kamil.kural/FdaPrecision Challenge/sc3_Phase1_CN_GE_Phenotype.tsv', sep='\t')

In [10]:
df = pd.merge(df2,
                    df1,
                    left_index=True,
                    right_index=True)
df = pd.merge(df,
                    df3,
                    left_index=True,
                    right_index=True)

In [11]:
df=df.drop(['PATIENTID','PATIENTID_x','PATIENTID_y'], axis=1)

In [12]:
dfnew=df[['PCMTD1', 'ARAP3','DYNLL2' ,'HAUS4', 'C4orf19', 'SMARCC2', 'CASTOR1' ,'PDCD4','SYCP2', 'ACBD4', 'SMU1' ,'SETD1A', '11p15.4', '9p24.1', '9p21.3' ,'20q11.21']]

In [13]:
# dfnew= PCMTD1
# ARAP3
# DYNLL2
# HAUS4
# C4orf19
# SMARCC2
# CASTOR1
# are the important features coming from Subchallenge 1.

# The important features coming from Subchallenge 2 are 
# 11p15.4
# 9p24.1
# 9p21.3
# 20q11.21

In [18]:
dfnew.shape

(166, 16)

In [19]:
all_features = [x for x in dfnew.columns if x not in ['SURVIVAL_STATUS']]

In [20]:
X, y = dfnew[all_features], df['SURVIVAL_STATUS']
labelencoder_y_1 = LabelEncoder()
y = labelencoder_y_1.fit_transform(y)
y

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [21]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42,stratify=df['SURVIVAL_STATUS'])

In [23]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

In [24]:
pbounds = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100,200),
    'max_depth': (15,25),
    'subsample': (0.1, 1.0),  # Change for big datasets
    'colsample': (0.1, 1.0),  # Change for datasets with lots of features
    'gamma': (0, 5),
    'max_delta_step' : (0,5),
    'min_child_weight' : (0,5),
    'reg_alpha' : (0.1, 1),
    'reg_lambda' : (0.1,1),
    'scale_pos_weight' : (0.1, 1)}
 
def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        max_depth,
                        subsample,
                        colsample,
                        gamma,
                        max_delta_step,
                        min_child_weight,
                        reg_alpha,
                        reg_lambda,
                        scale_pos_weight):
 
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
 
    clf = XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        gamma=gamma,
        max_delta_step=max_delta_step,
        min_child_weight=min_child_weight,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        scale_pos_weight=scale_pos_weight)
    return np.mean(cross_val_score(clf, X, y, cv=10, scoring='accuracy',n_jobs=50))
 
 
optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=pbounds,
    random_state=1)

In [25]:
optimizer.maximize(init_points=10, n_iter=20, acq='ei')

|   iter    |  target   | colsample |   gamma   | learni... | max_de... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.796    |  0.4753   |  3.602    |  0.01011  |  1.512    |  16.47    |  0.4617   |  118.6    |  0.411    |  0.4571   |  0.5849   |  0.4773   |
|  2        |  0.8199   |  0.7167   |  1.022    |  0.8793   |  0.1369   |  21.7     |  2.087    |  155.9    |  0.2263   |  0.2783   |  0.8207   |  0.9714   |
|  3        |  0.7544   |  0.3821   |  3.462    |  0.8776   |  4.473    |  15.85    |  0.1953   |  117.0    |  0.8903   |  0.1885   |  0.479    |  0.9621   |
|  4        |  0.7846   |  0.5798   |  3.459    |  0.3224   |  3.433    |  23.35    |  0.09144  |  175.0    |  0.99     |  0.7733   |  0.3524   |  0.8104   |
|  5        |  0.7493   |  0.1929   |  2.239    |  0

In [26]:
optimizer.max

{'target': 0.8198529411764707,
 'params': {'colsample': 0.7166975503570835,
  'gamma': 1.0222612486575873,
  'learning_rate': 0.879336262027036,
  'max_delta_step': 0.13693796598963082,
  'max_depth': 21.704675101784023,
  'min_child_weight': 2.086524011835635,
  'n_estimators': 155.86898284457516,
  'reg_alpha': 0.2263482447357104,
  'reg_lambda': 0.2782913401763909,
  'scale_pos_weight': 0.820670111807983,
  'subsample': 0.9714354181474578}}

In [47]:
model1=XGBClassifier(base_score=0.5, booster='gbtree', colsample=1,
              gamma=1,
              learning_rate=0.01, max_delta_step=0, max_depth=21,
              min_child_weight=2, missing=None, n_estimators=200, n_jobs=20,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0.1, reg_lambda=0.1, scale_pos_weight=0.4, seed=None,
              silent=None, subsample=1, verbosity=1)

In [48]:
eval_set = [(X_train, y_train), (X_test, y_test)]
eval_metric = ["auc","error"]
%time model1.fit(X_train,y_train,eval_metric =eval_metric, eval_set=eval_set, verbose=False)

CPU times: user 1.2 s, sys: 4.02 ms, total: 1.2 s
Wall time: 61.4 ms


XGBClassifier(base_score=0.5, booster='gbtree', colsample=1,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=1, learning_rate=0.01, max_delta_step=0, max_depth=21,
              min_child_weight=2, missing=None, n_estimators=200, n_jobs=20,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0.1, reg_lambda=0.1, scale_pos_weight=0.4, seed=None,
              silent=None, subsample=1, verbosity=1)

In [49]:
pred=model1.predict(X)

In [50]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
tn, fp, fn, tp = confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
cm=confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
total=sum(cm)
accuracy=(tp+tn)/(tp+tn+fn+fp)
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
print('Specificity : ', specificity)
print('Sensitivity : ', sensitivity )
print ('Accuracy : ', accuracy)
print('Confusion Matrix : \n', cm)

Specificity :  0.875
Sensitivity :  0.9206349206349206
Accuracy :  0.9096385542168675
Confusion Matrix : 
 [ 35   5  10 116]


In [51]:
from sklearn.ensemble import RandomForestClassifier
pbounds = {
    
    'n_estimators': (100,500),
    'max_depth': (20, 500),  # Change for big datasets
    'min_samples_split': (0.1, 1),  # Change for datasets with lots of features
                                }
 
def rf_hyper_param(
                        n_estimators,
                        max_depth,
                        min_samples_split,
                        
                        ):
 
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
 
    clf = RandomForestClassifier(
        max_depth=max_depth,
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        
        )
    return np.mean(cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc',n_jobs=50))
 
 
optimizer = BayesianOptimization(
    f=rf_hyper_param,
    pbounds=pbounds,
    random_state=1)

In [52]:
optimizer.maximize(init_points=10, n_iter=20, acq='ei')

|   iter    |  target   | max_depth | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        |  0.5      |  220.2    |  0.7483   |  100.0    |
|  2        |  0.8098   |  165.1    |  0.2321   |  136.9    |
|  3        |  0.8088   |  109.4    |  0.411    |  258.7    |
|  4        |  0.8114   |  278.6    |  0.4773   |  374.1    |
|  5        |  0.5      |  118.1    |  0.8903   |  111.0    |
|  6        |  0.814    |  341.8    |  0.4756   |  323.5    |
|  7        |  0.8148   |  87.39    |  0.2783   |  420.3    |
|  8        |  0.8031   |  484.8    |  0.3821   |  376.9    |
|  9        |  0.5      |  440.7    |  0.9051   |  134.0    |
|  10       |  0.8169   |  38.75    |  0.2528   |  451.3    |
|  11       |  0.8124   |  200.0    |  0.1      |  219.4    |
|  12       |  0.816    |  369.6    |  0.1      |  500.0    |
|  13       |  0.8143   |  183.2    |  0.1      |  500.0    |
|  14       |  0.5      |  500.0    |  1.0      |  500.0    |
|  15   

In [53]:
randomforest = RandomForestClassifier(criterion='gini',n_estimators=317,max_depth=441,min_samples_split=0.10,min_impurity_decrease=0.0,min_weight_fraction_leaf=0.0,max_features='auto', min_samples_leaf=8,random_state=0,oob_score=True,class_weight='balanced_subsample')

In [54]:
randomforest.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
                       criterion='gini', max_depth=441, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=8,
                       min_samples_split=0.1, min_weight_fraction_leaf=0.0,
                       n_estimators=317, n_jobs=None, oob_score=True,
                       random_state=0, verbose=0, warm_start=False)

In [55]:
pred=randomforest.predict(X)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
tn, fp, fn, tp = confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
cm=confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
total=sum(cm)
accuracy=(tp+tn)/(tp+tn+fn+fp)
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
print('Specificity : ', specificity)
print('Sensitivity : ', sensitivity )
print ('Accuracy : ', accuracy)
print('Confusion Matrix : \n', cm)

Specificity :  0.825
Sensitivity :  0.8809523809523809
Accuracy :  0.8674698795180723
Confusion Matrix : 
 [ 33   7  15 111]


In [56]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
clf = GridSearchCV(LogisticRegression(penalty='l2'), param_grid)
GridSearchCV(cv=5,
             estimator=LogisticRegression(C=1.0, intercept_scaling=1, class_weight='balanced' ,
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001,n_jobs=50,verbose= 1000),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                          dual=False, fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=50, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=1000,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [57]:
clf.fit(X_train,y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [58]:
best_params = clf.best_params_
best_params

{'C': 0.1}

In [68]:
lr1=LogisticRegression(C=0.01, class_weight='balanced', dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=300, multi_class='ovr',
                                          n_jobs=20, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=1e-4, verbose=1,
                                          warm_start=False)

In [69]:
lr1.fit(X_train,y_train)

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 out of   1 | elapsed:    0.5s finished


LogisticRegression(C=0.01, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=300, multi_class='ovr', n_jobs=20, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=1,
                   warm_start=False)

In [70]:
pred=lr1.predict(X)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
tn, fp, fn, tp = confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
cm=confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
total=sum(cm)
accuracy=(tp+tn)/(tp+tn+fn+fp)
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
print('Specificity : ', specificity)
print('Sensitivity : ', sensitivity )
print ('Accuracy : ', accuracy)
print('Confusion Matrix : \n', cm)

Specificity :  0.75
Sensitivity :  0.7936507936507936
Accuracy :  0.7831325301204819
Confusion Matrix : 
 [ 30  10  26 100]


In [71]:
def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
    params = {'application':'binary','num_iterations':300, 'learning_rate':0.05, 'early_stopping_round':100, 'metric':'auc'}
    params["num_leaves"] = round(num_leaves)
    params['feature_fraction'] = max(min(feature_fraction, 1), 0)
    params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
    params['max_depth'] = round(max_depth)
    params['lambda_l1'] = max(lambda_l1, 0)
    params['lambda_l2'] = max(lambda_l2, 0)
    params['min_split_gain'] = min_split_gain
    params['min_child_weight'] = min_child_weight
    cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
    return max(cv_result['auc-mean'])

In [72]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                        'feature_fraction': (0.1, 0.9),
                                        'bagging_fraction': (0.8, 1),
                                        'max_depth': (5, 8.99),
                                        'lambda_l1': (0, 5),
                                        'lambda_l2': (0, 3),
                                        'min_split_gain': (0.001, 0.1),
                                        'min_child_weight': (5, 50)}, random_state=0)

In [73]:
def bayes_parameter_opt_lgb(X_train, y_train, init_round=15, opt_round=25, n_folds=5, random_seed=0, n_estimators=10000, learning_rate=0.05, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=X_train, label=y_train)
    # parameters
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight,n_jobs=20):
        params = {'application':'binary','num_iterations': n_estimators, 'learning_rate':learning_rate, 'early_stopping_round':100, 'metric':'auc'}
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return max(cv_result['auc-mean'])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.8, 1),
                                            'max_depth': (5, 25),
                                            'lambda_l1': (0, 5),
                                            'lambda_l2': (0, 3),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (1, 10)}, random_state=0)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")
    
    # return best parameters
    

opt_params = bayes_parameter_opt_lgb(X_train, y_train, init_round=10, opt_round=20, n_folds=5, random_seed=0, n_estimators=100, learning_rate=0.01)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7189   |  0.9098   |  0.6722   |  3.014    |  1.635    |  13.47    |  6.813    |  0.04432  |  42.73    |
|  2        |  0.7439   |  0.9927   |  0.4068   |  3.959    |  1.587    |  16.36    |  9.33     |  0.008033 |  25.83    |
|  3        |  0.7052   |  0.804    |  0.7661   |  3.891    |  2.61     |  24.57    |  8.192    |  0.04669  |  40.39    |
|  4        |  0.7232   |  0.8237   |  0.6119   |  0.7168   |  2.834    |  15.44    |  4.732    |  0.02719  |  40.26    |
|  5        |  0.7233   |  0.8912   |  0.5547   |  0.09395  |  1.853    |  17.24    |  6.552    |  0.09443  |  38.32    |
|  6        |  0.7396   |  0.8719   |  0.4496   |  3.488    |  0.1807   |  18.34    |  7.036    |  0.02183  |  26.71    |
|  7        |  0.7699   

In [74]:
from lightgbm import LGBMClassifier
model2=LGBMClassifier(num_leaves= 24, feature_fraction= 0.1, learning_rate= 0.01, bagging_fraction= 1, max_depth= 5,scale_pos_weight=0.5, min_child_weight= 1, n_estimators= 200, lambda_l1= 0, lambda_l2= 0, min_split_gain=0.1, n_jobs= 20, verbose=1)

In [75]:
model2.fit(X_train,y_train,eval_metric =eval_metric, eval_set=eval_set, verbose=True)

[1]	training's auc: 0.706406	training's binary_logloss: 0.552482	valid_1's auc: 0.778846	valid_1's binary_logloss: 0.544137
[2]	training's auc: 0.873437	training's binary_logloss: 0.549609	valid_1's auc: 0.831731	valid_1's binary_logloss: 0.541677
[3]	training's auc: 0.926094	training's binary_logloss: 0.546984	valid_1's auc: 0.829327	valid_1's binary_logloss: 0.540681
[4]	training's auc: 0.93125	training's binary_logloss: 0.544528	valid_1's auc: 0.838942	valid_1's binary_logloss: 0.538528
[5]	training's auc: 0.938281	training's binary_logloss: 0.542528	valid_1's auc: 0.848558	valid_1's binary_logloss: 0.537448
[6]	training's auc: 0.937969	training's binary_logloss: 0.540066	valid_1's auc: 0.90625	valid_1's binary_logloss: 0.534215
[7]	training's auc: 0.940312	training's binary_logloss: 0.537868	valid_1's auc: 0.877404	valid_1's binary_logloss: 0.533127
[8]	training's auc: 0.937187	training's binary_logloss: 0.53632	valid_1's auc: 0.882212	valid_1's binary_logloss: 0.531914
[9]	trainin

LGBMClassifier(bagging_fraction=1, boosting_type='gbdt', class_weight=None,
               colsample_bytree=1.0, feature_fraction=0.1,
               importance_type='split', lambda_l1=0, lambda_l2=0,
               learning_rate=0.01, max_depth=5, min_child_samples=20,
               min_child_weight=1, min_split_gain=0.1, n_estimators=200,
               n_jobs=20, num_leaves=24, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, scale_pos_weight=0.5, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
               verbose=1)

In [76]:
pred=model2.predict(X)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
tn, fp, fn, tp = confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
cm=confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
total=sum(cm)
accuracy=(tp+tn)/(tp+tn+fn+fp)
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
print('Specificity : ', specificity)
print('Sensitivity : ', sensitivity )
print ('Accuracy : ', accuracy)
print('Confusion Matrix : \n', cm)

Specificity :  0.7
Sensitivity :  0.9523809523809523
Accuracy :  0.891566265060241
Confusion Matrix : 
 [ 28  12   6 120]


In [77]:
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.82      0.70      0.76        40
           1       0.91      0.95      0.93       126

    accuracy                           0.89       166
   macro avg       0.87      0.83      0.84       166
weighted avg       0.89      0.89      0.89       166



In [87]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
#List Hyperparameters that we want to tune.
n_neighbors = list(range(1,30))
p=[1,2,3,4,5,6,7,8,9,10]
#Convert to dictionary
hyperparameters = dict( n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10, n_jobs=50,verbose=1)
#Fit the model
best_model = clf.fit(X,y)
#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

Fitting 10 folds for each of 290 candidates, totalling 2900 fits


[Parallel(n_jobs=50)]: Using backend LokyBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done 100 tasks      | elapsed:    1.1s
[Parallel(n_jobs=50)]: Done 600 tasks      | elapsed:    1.5s


Best leaf_size: 30
Best p: 4
Best n_neighbors: 10


[Parallel(n_jobs=50)]: Done 2900 out of 2900 | elapsed:    2.3s finished


In [91]:
knn = KNeighborsClassifier(leaf_size=30, p=4, n_neighbors= 10,weights='distance')

In [92]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=4,
                     weights='distance')

In [93]:
pred=knn.predict(X)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
tn, fp, fn, tp = confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
cm=confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
total=sum(cm)
accuracy=(tp+tn)/(tp+tn+fn+fp)
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
print('Specificity : ', specificity)
print('Sensitivity : ', sensitivity )
print ('Accuracy : ', accuracy)
print('Confusion Matrix : \n', cm)

Specificity :  0.925
Sensitivity :  0.9841269841269841
Accuracy :  0.9698795180722891
Confusion Matrix : 
 [ 37   3   2 124]


In [95]:
roc_auc_score(df['SURVIVAL_STATUS'], pred)

0.9545634920634921

In [94]:
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94        40
           1       0.98      0.98      0.98       126

    accuracy                           0.97       166
   macro avg       0.96      0.95      0.96       166
weighted avg       0.97      0.97      0.97       166



In [98]:
# from sklearn.linear_model import RidgeClassifier
# develop = True
# seed = 0

    
# def target(**params):
#     fit_intercept = int(params['fit_intercept'])
#     fit_intercept_dict = {0:False, 1:True}

#     model = RidgeClassifier(alpha = params['alpha'],
#                     fit_intercept = fit_intercept_dict[fit_intercept],
#                     copy_X = True,class_weight='balanced' )
    
#     scores = cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=5)
#     return scores.mean()
    
# params = {'alpha':(1, 50),
#           'fit_intercept':(0,1.99)}
# if develop:
#     bo = BayesianOptimization(target, params, random_state=seed)
#     bo.maximize(init_points=5, n_iter=10, acq='ucb')
    

|   iter    |  target   |   alpha   | fit_in... |
-------------------------------------------------
|  1        |  0.7148   |  27.89    |  1.423    |
|  2        |  0.7148   |  30.54    |  1.084    |
|  3        |  0.7148   |  21.76    |  1.285    |
|  4        |  0.7148   |  22.44    |  1.775    |
|  5        |  0.595    |  48.22    |  0.763    |
|  6        |  0.7131   |  1.0      |  1.99     |
|  7        |  0.5936   |  1.0      |  3.117e-0 |
|  8        |  0.7131   |  9.778    |  1.99     |
|  9        |  0.595    |  39.16    |  0.0      |
|  10       |  0.7131   |  15.82    |  1.99     |
|  11       |  0.595    |  28.99    |  0.0      |
|  12       |  0.7148   |  41.79    |  1.99     |
|  13       |  0.7148   |  35.56    |  1.99     |
|  14       |  0.5936   |  13.28    |  0.0      |
|  15       |  0.7131   |  4.952    |  1.99     |


In [99]:
# ridgemodel = RidgeClassifier(alpha = 22.44,
#                   fit_intercept = 1.45, class_weight='balanced',
#                   copy_X = True)

# ridgemodel.fit(X_train, y_train)
# pred = ridgemodel.predict(X)
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report
# tn, fp, fn, tp = confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
# cm=confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
# total=sum(cm)
# accuracy=(tp+tn)/(tp+tn+fn+fp)
# sensitivity = tp/(tp+fn)
# specificity = tn/(tn+fp)
# print('Specificity : ', specificity)
# print('Sensitivity : ', sensitivity )
# print ('Accuracy : ', accuracy)
# print('Confusion Matrix : \n', cm)

Specificity :  0.85
Sensitivity :  0.9841269841269841
Accuracy :  0.9518072289156626
Confusion Matrix : 
 [ 34   6   2 124]


In [108]:
from sklearn.ensemble import VotingClassifier

voting_classifier = VotingClassifier(estimators=[

    ('xgb',model1), 
    ('knn', knn)],voting='hard')
voting_classifier.fit(X_train,y_train)

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample=1, colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=1,
                                            learning_rate=0.01,
                                            max_delta_step=0, max_depth=21,
                                            min_child_weight=2, missing=None,
                                            n_estimators=200, n_jobs=20,
                                            nthread=None,
                                            objective='binary:logistic',
                                            random_state=0, reg_alpha=0.1,
                                            reg_lambda=0.1,
                                            scale_pos_weight=0.4, seed=None,
                                    

In [109]:
pred=voting_classifier.predict(X)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
tn, fp, fn, tp = confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
cm=confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
total=sum(cm)
accuracy=(tp+tn)/(tp+tn+fn+fp)
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
print('Specificity : ', specificity)
print('Sensitivity : ', sensitivity )
print ('Accuracy : ', accuracy)
print('Confusion Matrix : \n', cm)

Specificity :  0.95
Sensitivity :  0.9126984126984127
Accuracy :  0.9216867469879518
Confusion Matrix : 
 [ 38   2  11 115]


In [98]:
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.69      0.90      0.78        40
           1       0.96      0.87      0.92       126

    accuracy                           0.88       166
   macro avg       0.83      0.89      0.85       166
weighted avg       0.90      0.88      0.88       166



In [114]:
from mlxtend.classifier import StackingCVClassifier
lr = LogisticRegression(C=0.1)
sclf = StackingCVClassifier(classifiers = [lr1, randomforest, model1, knn],
                            shuffle = False,
                            use_probas = True,
                            cv = 5,
                            meta_classifier = lr)

In [115]:
sclf.fit(X_train, y_train)

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 out of   1 | elapsed:    0.5s finished


StackingCVClassifier(classifiers=[LogisticRegression(C=0.01,
                                                     class_weight='balanced',
                                                     dual=False,
                                                     fit_intercept=True,
                                                     intercept_scaling=1,
                                                     l1_ratio=None,
                                                     max_iter=300,
                                                     multi_class='ovr',
                                                     n_jobs=20, penalty='l2',
                                                     random_state=None,
                                                     solver='lbfgs', tol=0.0001,
                                                     verbose=1,
                                                     warm_start=False),
                                  RandomForestClassifier(bootstrap=True,
  

In [116]:
pred=sclf.predict(X)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
tn, fp, fn, tp = confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
cm=confusion_matrix(df['SURVIVAL_STATUS'], pred,labels=[0,1]).ravel()
total=sum(cm)
accuracy=(tp+tn)/(tp+tn+fn+fp)
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
print('Specificity : ', specificity)
print('Sensitivity : ', sensitivity )
print ('Accuracy : ', accuracy)
print('Confusion Matrix : \n', cm)

Specificity :  0.625
Sensitivity :  1.0
Accuracy :  0.9096385542168675
Confusion Matrix : 
 [ 25  15   0 126]


In [117]:
roc_auc_score(df['SURVIVAL_STATUS'], pred)

0.8125

In [103]:
print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       1.00      0.62      0.77        40
           1       0.89      1.00      0.94       126

    accuracy                           0.91       166
   macro avg       0.95      0.81      0.86       166
weighted avg       0.92      0.91      0.90       166

